In [1]:
from swf import *
from utils import *
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit,lsq_linear,brute
from scipy.sparse.linalg import LinearOperator
from scipy.stats import norm
import plotly.express as px
import pandas as pd

In [2]:
octvertices = np.array([[1,0,0],[-1,0,0],[0,1,0],[0,-1,0],[0,0,1],[0,0,-1]])
octfaces = np.array([[1,2,4],[1,3,4],[3,0,4],[0,2,4],[1,3,5],[3,0,5],[0,2,5],[2,1,5]])

layout704=np.array([[1,0,np.pi/2],[1,np.pi/6,np.pi/2],[1,-np.pi/6,np.pi/2],[1,np.pi/2,np.pi/2],[1,-np.pi/2,np.pi/2],[1,3*np.pi/4,np.pi/2],[1,-3*np.pi/4,np.pi/2],[1,np.pi/4,np.pi/4],[1,-np.pi/4,np.pi/4],[1,3*np.pi/4,np.pi/4],[1,-3*np.pi/4,np.pi/4]])
vertices704 = np.apply_along_axis(lambda x: toCartesian(x),1,layout704)
faces704 = np.array([[6,4,10],[10,4,8],[8,4,2],[8,2,0],[8,7,0],[7,0,1],[7,1,3],[7,3,9],[9,3,5],[10,7,9],[10,8,7],[10,6,5],[10,9,5]])

In [18]:
def optimize(coeffs):
    print('.',end='')
    ALPHA, BETA = coeffs
    GAMMA = (1-2*(ALPHA+BETA))/4
    if checkCoeffRelations(ALPHA,BETA,GAMMA):
        mesh = Trimesh(octvertices,octfaces,ALPHA=ALPHA,BETA=BETA,GAMMA=GAMMA)
        model = SWF(mesh, n=3)
        return cost(model,1,1)
    else:
        return 100

In [19]:
A = LinearOperator((1,2),optimize)
A

.

<1x2 _CustomLinearOperator with dtype=float64>

In [27]:
aDist = norm(loc=0.62, scale=0.01)
bDist = norm(loc=-0.01, scale=0.01)
aRVS = aDist.rvs(size=1200).reshape((-1,1))
bRVS = bDist.rvs(size=1200).reshape((-1,1))
v = np.hstack((aRVS,bRVS))
b = np.array([])
for i in range(1200):
    b = np.append(b,A.matvec(v[i]))

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [30]:
df = pd.DataFrame(np.hstack((v,b.reshape(-1,1))),columns=['a','b','cost'])
fig = px.scatter_3d(df[df.cost<0.0014], x="a", y="b", z='cost',color='cost')
fig.show()

In [29]:
df[df.cost ==df['cost'].min()]

,a,b,cost
745,0.609429,-0.015111,0.001298
